## Imports

In [ ]:
# install caso não tenha
# !python -m spacy download pt_core_news_sm

In [1]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

C:\Users\william.yizima\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Baixando Corpora:
- http://www.nilc.icmc.usp.br/embeddings
    - download cbow - 300 dimensões (fim da página)
    - download skig_gram - 300 dimensões (fim da página)
- Esse corpora serve de dicionário para a transformação de palavras em vetores

In [5]:
# Carregando os "dicionários"
w2v_modelo_cbow = KeyedVectors.load_word2vec_format("./corpora_nilc/cbow_s300.txt")

## Testando tokenizador

### Criando uma instância do Spacy

In [77]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

### Tokenizador
- Dividindo a frase em palavras de maneira mais assertiva
- tratando algumas palavras:
    - números - não trazem grande significância(semantica) a frase
    - palavras "inúteis" para a semântica da frase

In [109]:
frase_exemplo = 'Eu gosto do GOKU! -21/10/2000 '

In [110]:
## inserindo a frase na instância
doc = nlp(frase_exemplo)

for token in doc:
    print(f'token original: {token}')
    print(f'token tratado: {token.text.lower()}')
    print('~~~~ \n')

token original: Eu
token tratado: eu
~~~~ 

token original: gosto
token tratado: gosto
~~~~ 

token original: do
token tratado: do
~~~~ 

token original: GOKU
token tratado: goku
~~~~ 

token original: !
token tratado: !
~~~~ 

token original: -21/10/2000
token tratado: -21/10/2000
~~~~ 



In [111]:
for token in doc:
    if (not token.is_stop and token.is_alpha):
        print(f'token original: {token}')
        print(f'token tratado: {token.text.lower()}')
        print('~~~~ \n')

token original: gosto
token tratado: gosto
~~~~ 

token original: GOKU
token tratado: goku
~~~~ 



In [112]:
# Criando uma função
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

def tokenizador(texto):
    """
        recebe um texto e tokeniza, lowe_case, retira numeros e stop words
    """
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    
    return  tokens_validos

tokens = tokenizador(frase_exemplo)
print(tokens)

['gosto', 'goku']


### Transformando os tokens em vetores

In [122]:
def combinacao_de_vetores(palavras, modelo):
    """
        recebe palavras tokenizadas e um modelo para transformar em um vetor
    """
    vetor_resultante = np.zeros((1,300))

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass
                

    return vetor_resultante

vetor_texto = combinacao_de_vetores(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[ 0.15944     0.184455    0.047713   -0.05863401  0.17082501 -0.18356301
   0.203713   -0.011041    0.70918301 -0.12342701 -0.401985    0.063911
  -0.323165   -0.05355601 -0.15809899 -0.22740199 -0.41576599 -0.69300199
   0.159361    0.14347801 -0.14933201 -0.15896101 -0.262879    0.49546199
   0.34412001  0.046617    0.102513    0.177324   -0.37688401  0.12481798
  -0.000995   -0.42847899  0.220005   -0.026092    0.58421199 -0.171002
   0.10148299 -0.048152    0.258902    0.18021301  0.117406    0.10504
   0.045488    0.30392901 -0.04425     0.25195001 -0.18918901 -0.35875701
   0.423582   -0.07098     0.40591     0.037249   -0.331585    0.157816
   0.299494    0.22547299  0.46450201  0.19722299 -0.247617   -0.091273
  -0.161918    0.196182    0.38344501 -0.382522   -0.28660199 -0.532369
   0.25413899 -0.391881    0.12975501 -0.33665199 -0.234873   -0.7362
  -0.039475   -0.17230098 -0.01678099  0.23881     0.115316    0.28634
  -0.05727     0.015823   -0.27652601  0.12886801

In [117]:
def matriz_vetores(textos, modelo):
    """
        utilizando o combinação de textos para várias linhas do dataset
    """
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores(palavras, modelo)

    return matriz

In [118]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino):
    """
        Treinando um Classificador
    """
    RL = LogisticRegression(max_iter = 800)
    RL.fit(x_treino, y_treino)
    
    return RL

## Dataset
- Dataset extraído da base do blip de interações
- tratado com alguns exemplos mais úteis
- balanceado


In [9]:
dataset = pd.read_csv("./dataset_balanceado.csv")

### Tipos de intenção

In [127]:
qtd_classes_abs = dataset['Intenção'].value_counts()
qtd_classes_abs

FazerTroca     1121
None           1000
MeusPedidos     827
Name: Intenção, dtype: int64

### Exemplos

In [128]:
dataset[dataset['Intenção']=='MeusPedidos']

,Unnamed: 0,Dados,Frase do usuário,Intenção,sentenceWithoutStopWord
2121,0,2021-03-26 12:32:26 +00:00,acompanhar pedido,MeusPedidos,acompanhar pedido
2122,18,2021-04-16 23:25:47 +00:00,pedir reembolso,MeusPedidos,pedir reembolso
2123,46,2021-03-29 18:02:13 +00:00,tirar duvida,MeusPedidos,tirar duvida
2124,51,2021-04-03 00:34:09 +00:00,garantia de pedido,MeusPedidos,garantia pedido
2125,106,2021-04-09 03:06:00 +00:00,meu pedido ta sendo processado ainda e no meu ...,MeusPedidos,pedido sendo processado cartao consta
...,...,...,...,...,...
2943,1502,2021-04-16 12:42:47 +00:00,porque demora tanto esta aprovacao se logo qua...,MeusPedidos,demora aprovacao pagamento pedido bate pronto ...
2944,1512,2021-05-01 13:30:29 +00:00,esta com meu pedido para entrega atrasado,MeusPedidos,pedido entrega atrasado
2945,1519,2021-04-26 22:58:19 +00:00,recebi hoje o meu pedido,MeusPedidos,recebi hoje pedido
2946,1522,2021-03-15 15:00:51 +00:00,nao consigo concluir pedido,MeusPedidos,consigo concluir pedido


In [130]:
dataset[dataset['Intenção']=='FazerTroca']

,Unnamed: 0,Dados,Frase do usuário,Intenção,sentenceWithoutStopWord
1000,3,2021-04-07 20:03:48 +00:00,🔄trocas e devolucao,FazerTroca,trocas devolucao
1001,6,2021-04-07 17:49:38 +00:00,troca de produto,FazerTroca,troca produto
1002,13,2021-04-17 15:06:37 +00:00,defeito,FazerTroca,defeito
1003,24,2021-03-20 11:35:43 +00:00,🔄troca de produto,FazerTroca,troca produto
1004,27,2021-03-25 22:00:52 +00:00,quero informacao da prancha sensi 4 d,FazerTroca,informacao prancha sensi d
...,...,...,...,...,...
2116,1492,2021-02-17 20:22:47 +00:00,quero trocar a mercadoria,FazerTroca,trocar mercadoria
2117,1507,2021-05-17 15:42:15 +00:00,boa tarde gostaria de saber as medidas desse p...,FazerTroca,gostaria medidas produto carteira couro
2118,1514,2021-04-14 13:10:28 +00:00,preciso de uma copia da ultima nota fiscal,FazerTroca,preciso copia ultima nota fiscal
2119,1523,2021-03-09 15:35:49 +00:00,gostaria de saber se ha previsao de chegada do...,FazerTroca,gostaria ha previsao chegada tenis masculinos


In [131]:
dataset[dataset['Intenção']=='None']

,Unnamed: 0,Dados,Frase do usuário,Intenção,sentenceWithoutStopWord
0,7,2021-03-24 14:25:02 +00:00,"👍sim, confirmo",None,confirmo
1,11,2021-03-19 18:30:50 +00:00,dsvvsdvvsd,None,dsvvsdvvsd
2,12,2021-03-23 11:32:53 +00:00,nao foi entregue,None,entregue
3,14,2021-03-19 13:06:52 +00:00,prezados senhores,None,prezados senhores
4,40,2021-04-04 15:26:35 +00:00,gostaria de saber onde fica a assistencia tecn...,None,gostaria onde fica assistencia tecnica gama ci...
...,...,...,...,...,...
995,145,2021-05-17 13:28:33 +00:00,nao consigo recuperar minha senha,None,consigo recuperar senha
996,147,2021-05-15 00:57:36 +00:00,eu quero pijama de inverno,None,pijama inverno
997,151,2021-04-12 16:04:39 +00:00,essa jaqueta tem capuz,None,jaqueta capuz
998,164,2021-04-13 12:02:02 +00:00,tem calca capri de cotton?,None,calca capri cotton


## Separando meu  X e Y
- x -> frases do usuário (o que input para previsão)
- y -> intenção ( o que quero prever)

In [114]:
frases_treino = dataset['Frase do usuário']
classificacoes_treino = dataset['Intenção']

## Treinando

In [115]:
# transoframndo meu dataset de treino em vetores
matriz_vetores_treino_cbow = matriz_vetores(frases_treino, w2v_modelo_cbow)

In [119]:
# Treinando
RL_cbow = classificador(w2v_modelo_cbow,
                        matriz_vetores_treino_cbow,
                        classificacoes_treino)

## Testando

In [120]:
sentence = 'nao consigo concluir pedido'

In [123]:
# fazendo um exemplo
titulo_tokens = tokenizador(sentence)
titulo_vetor = combinacao_de_vetores(titulo_tokens, w2v_modelo_cbow)
titulo_categoria = RL_cbow.predict(titulo_vetor)

In [132]:
titulo_categoria[0]

'MeusPedidos'

## Salvando em um binário

In [133]:
import pickle
pickle.dump(RL_cbow, open('modelo_nlp.sav','wb'))

In [134]:
modelo_carregado = pickle.load(open('modelo_nlp.sav','rb'))

In [136]:
titulo_vetor_modelo_carregado = combinacao_de_vetores(titulo_tokens, w2v_modelo_cbow)
titulo_categoria_modelo_carregado = modelo_carregado.predict(titulo_vetor_modelo_carregado)
titulo_categoria_modelo_carregado[0]

'MeusPedidos'